For this assignment, you will replicate the supplier selection analysis we covered in class on a new dataset. Specifically, in this dataset you will be evaluating suppliers of refridgerated warehouse services.

The following code block imports some necessary libraries.

In [8]:
import pathlib

import re

import numpy as np
import pandas as pd

from sklearn import preprocessing

import OM527_functions as omf

The following code block defines the `custom_grouper` function that we used in the notebook covering the concept of spend analysis.

In [9]:
def custom_grouper(df, agg_dict, groupby_columns):
    '''
    This function groups the provided DataFrame, df, by the columns
    specified in the groupby_columns argument. The aggregations specified
    in the agg_dict dictionary are applied. Also, each numeric column in the 
    aggregated DataFrame is used to create a proportion column. The aggregated data
    is returned as a DataFrame sorted by the keys of the agg_dict
    dictionary, in the order they are specified, i.e., first key
    has a higher sort priority than the second, etc...
    '''
    
    grouped_df = df.groupby(groupby_columns).agg(agg_dict)
    
    grouped_df.columns = ['_'.join(col).strip() for col in grouped_df.columns.values]
    
    numeric_columns = grouped_df.select_dtypes(include='number').columns.tolist()

    for column in numeric_columns:
        grouped_df[f'{column}_proportion'] = (grouped_df[column]/grouped_df[column].sum())
        
    grouped_df = grouped_df.sort_values(numeric_columns)

    return grouped_df

1) Read in the supplier data stored in the file `rw_supplier_data.csv` and print the first five rows. **(5 points)**

In [37]:
wb = pd.read_excel("data/OM_527_WCE_Questions.xlsx")

wb.head()

,Alternative,Credit Rating,Location,Delivery Guarantee,Annual Revenue
0,Alternative 1,B,Alabama,+/- 6 Hours,1015650.65
1,Alternative 2,B-,Alabama,+/- 24 Hours,1023218.10
2,Alternative 3,B-,Alabama,+/- 6 Hours,940268.39
3,Alternative 4,B,Georgia,+/- 6 Hours,976207.83
4,Alternative 5,B,Alabama,+/- 6 Hours,857593.91


# Task 1.1

In [38]:
Credit_Dict = {
    'A+' : 1,
    'A' : 0.98,
    'A-' : 0.95,
    'B+' : 0.9,
    'B' : 0.8,
    'B-' : 0.7,
    'C+' : 0.6,
    'C' : 0.5,
    'C-' : 0.0,
}

Locat_dict = {
    'Alabama' : 1,
    'Tennessee' : 0.4,
    'Georgia' : 0.4,
    'Mississippi' : 0.4,
    
    
}
Delivery_Dict = {
    '+/- 6 Hours' : 1,
    '+/- 24 Hours' : 0.8,
    np.nan : 0.2,
}

weights = {
    'Credit Rating' : 0.125,
    'Location' : 0.25,
    'Delivery Guarantee' : 0.375,
    'Annual Revenue criteria' : 0.125,
    
}

wb['Credit Rating'] = wb['Credit Rating'].map(Credit_Dict)
wb['Location'] = wb['Location'].map(Locat_dict)
wb['Delivery Guarantee'] = wb['Delivery Guarantee'].map(Delivery_Dict)

wb.nlargest(100, 'Credit Rating')

,Alternative,Credit Rating,Location,Delivery Guarantee,Annual Revenue
99,Alternative 100,1.00,1.0,0.2,968770.77
97,Alternative 98,1.00,1.0,0.2,910908.49
34,Alternative 35,1.00,1.0,1.0,881114.07
16,Alternative 17,1.00,0.4,1.0,986711.94
47,Alternative 48,0.98,0.4,1.0,1039009.33
...,...,...,...,...,...
8,Alternative 9,0.00,1.0,0.8,884381.76
72,Alternative 73,0.00,1.0,1.0,960055.10
70,Alternative 71,0.00,1.0,0.2,1009772.50
52,Alternative 53,0.00,1.0,0.8,796931.55


# Task 1.2

In [44]:
wb['WP'] = omf.mcdm.compute_weighted_product(wb, weights)

wb.nlargest(5, 'WP')

#wb.nlargest(10, 'WP')

,Alternative,Credit Rating,Location,Delivery Guarantee,Annual Revenue,WP
34,Alternative 35,1.00,1.0,1.0,881114.07,1.000000
24,Alternative 25,0.98,1.0,1.0,1052106.49,0.997478
60,Alternative 61,0.98,1.0,1.0,895474.66,0.997478
61,Alternative 62,0.98,1.0,1.0,1121114.53,0.997478
69,Alternative 70,0.98,1.0,1.0,1113689.14,0.997478
87,Alternative 88,0.98,1.0,1.0,926932.22,0.997478
30,Alternative 31,0.95,1.0,1.0,927440.26,0.993609
59,Alternative 60,0.95,1.0,1.0,1060884.38,0.993609
85,Alternative 86,0.95,1.0,1.0,1107774.38,0.993609
95,Alternative 96,0.95,1.0,1.0,918700.70,0.993609


# Task 1.3

The weighted product method is a lot more harsh on a particular category and you can immediately disqualify someone if they have a 0 in any 1 category (0 raised to a weight is still 0 then you multiply it through). This could be intentional if credit_score really should disqualify someone immediately.

# Task 1.4

The best method would be to take an ensemble approach, although a meeting with the relevant stakeholders could also be a good option to determine if there really should be a value that could result in immediate disqualification. There is an Ensemble Method that is shown in the workbook. The general lack of variability would indicate it is probably not needed as there is stiff competition for the top spot, but if the Alternatives were to decrease quality it would be worth considering a revised ranking method or a potential revision of the weights with input from relevant stakeholders on the selection of the alternatives.

In [43]:
ranking_methods_dict = {
    'WS': omf.mcdm.compute_weighted_sum,
    'WP': omf.mcdm.compute_weighted_product
}

omf.mcdm.robust_ranking(
    data=wb, 
    weights_dict=weights, 
    index_column="Alternative", 
    ranking_methods_dict=ranking_methods_dict, 
    perturbations=100, 
    top_values=10, 
    perturbation_range=0.3
)

,Alternative,WS,WP
24,Alternative 25,1.0,1.0
30,Alternative 31,1.0,1.0
34,Alternative 35,1.0,1.0
59,Alternative 60,1.0,1.0
60,Alternative 61,1.0,1.0
61,Alternative 62,1.0,1.0
69,Alternative 70,1.0,1.0
85,Alternative 86,1.0,1.0
87,Alternative 88,1.0,1.0
95,Alternative 96,1.0,1.0


# Task 2 Data

In [31]:
wb2 = pd.read_excel("data/OM_527_WCE_Questions.xlsx", "Spend_Analysis")

wb2.head()

,Transaction,Purchasing Department,Vendor,Spend
0,Transaction 1,Engineering,Vendor6,25287.96
1,Transaction 2,Engineering,Vendor1,16902.10
2,Transaction 3,Engineering,Vendor5,18078.90
3,Transaction 4,Engineering,Vendor7,11263.01
4,Transaction 5,Engineering,Vendor1,20126.08


# Task 2

In [33]:
groupby_columns2 = ['Purchasing Department']

agg_dict2 = {
    'Spend':['sum'],
}

data2 = custom_grouper(wb2, agg_dict2, groupby_columns2)


data2.sort_values(by = 'Spend_sum', ascending=False)



,Spend_sum,Spend_sum_proportion
Purchasing Department,,
Engineering,6172287.14,0.349978
Technology,3751742.80,0.212730
Legal,3157813.25,0.179053
Facilities,2703505.13,0.153293
Finance,1296635.85,0.073521
HR,554219.38,0.031425


Given 

In [36]:
groupby_columns2 = ['Vendor']

agg_dict2 = {
    'Purchasing Department':['nunique'],
    #'Spend':['sum'],
}

data2b = custom_grouper(wb2, agg_dict2, groupby_columns2)

data2b.sort_values(by = 'Spend_sum', ascending=False)

,Purchasing Department_nunique,Spend_sum,Purchasing Department_nunique_proportion,Spend_sum_proportion
Vendor,,,,
Vendor7,6,3392618.94,0.113208,0.192367
Vendor5,6,3053892.65,0.113208,0.173160
Vendor4,6,2379027.34,0.113208,0.134895
Vendor6,6,2018785.95,0.113208,0.114468
Vendor8,6,2017563.75,0.113208,0.114399
Vendor3,6,1898104.25,0.113208,0.107625
Vendor9,6,1484515.80,0.113208,0.084174
Vendor1,6,955991.38,0.113208,0.054206
Vendor2,5,435703.49,0.094340,0.024705


# Task 2.C

In [46]:
groupby_columns2 = ['Vendor']

agg_dict2 = {
    'Purchasing Department':['nunique'],
    'Spend':['sum'],
}

data2c = custom_grouper(wb2, agg_dict2, groupby_columns2)

data2c.sort_values(by = 'Spend_sum', ascending=False)


# Gross to look at, but this was also for me to see the per vendor spend ranking for each department


# groupby_columns2 = ['Purchasing Department', 'Vendor']

# agg_dict2 = {
#     'Spend':['sum'],
# }

# data2w = custom_grouper(wb2, agg_dict2, groupby_columns2)


# data2w.sort_values(by = 'Spend_sum', ascending=False)

,Purchasing Department_nunique,Spend_sum,Purchasing Department_nunique_proportion,Spend_sum_proportion
Vendor,,,,
Vendor7,6,3392618.94,0.113208,0.192367
Vendor5,6,3053892.65,0.113208,0.173160
Vendor4,6,2379027.34,0.113208,0.134895
Vendor6,6,2018785.95,0.113208,0.114468
Vendor8,6,2017563.75,0.113208,0.114399
Vendor3,6,1898104.25,0.113208,0.107625
Vendor9,6,1484515.80,0.113208,0.084174
Vendor1,6,955991.38,0.113208,0.054206
Vendor2,5,435703.49,0.094340,0.024705


# Task 3

     Analytic Hierarchy Process (AHP)

The AHP process is a method to do pairwise comparisons to help determine what the weights for a MCDM type operation would be.